<a href="https://colab.research.google.com/github/eruru0411/git-test/blob/master/dacon_final1111(%EC%B5%9C%EA%B3%A0%EC%A0%90%EC%88%98_1_07384)(%EC%9B%90%EB%B3%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import

In [ ]:
# 1. EDA  : 데이터 사전 탐색
#    0) Dataframe 살펴보기
#    1) 차트그리기
# 2. 결측치/이상치 --> 처리
# 3. 피쳐 전처리/가공
# 4. 학습/평가 : f1, auc
# 5. 검증(GridSearchCV, confusion_matrix)

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping

import pandas  as pd
import numpy   as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import tensorflow as tf

from scipy            import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import confusion_matrix, accuracy_score, f1_score, roc_auc_score,\
                             precision_score, recall_score, roc_curve, classification_report, precision_recall_curve
from sklearn                 import preprocessing
from sklearn.preprocessing   import StandardScaler,RobustScaler,MinMaxScaler, Binarizer, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, KFold
import warnings
warnings.filterwarnings(action="ignore")
barline = "\n============================================"
e = "\n"

# Dataframe 확인 ---------------------------------------
def MyCheckDataframe(df, target="target") :
    print("Check Dataframe ----------------------------")
    print("Shape :", e, df.shape)
    print("Info  :", e, df.info())
    print("Head  :", e, df.head())
    print("Count :", e, df[target].value_counts())

    y = df[target]
    X = df.drop(target, axis=1)
    print("Target Shape :", e, X.shape, y.shape, barline)
    return X, y
# X,y = CHECK_DATAFRAME(df, target="")


# -------------------------------------
# 4-2. IQR : 25%~75% 범위 값
# -------------------------------------
def MyCheckOutlier(df=None, column=None, print=None):
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    for k, v in df.items():
        Q1 = v.quantile(0.25)
        Q3 = v.quantile(0.75)
        IQR = Q3 - Q1
        IQR_weight = IQR * 1.5
        minimum = Q1 - IQR_weight
        maximum = Q3 + IQR_weight
        outlier_idx = v[(v < minimum) | (v> maximum)].index

        if print == True:
            perc = np.shape(outlier_idx)[0] * 100.0 / np.shape(df)[0]
            print("Column %s outliers = %.2f%%" % (k, perc))

    print(barline)
    return outlier_idx
    # Q1  = np.percentile(df[column].values, 25)
    # Q3  = np.percentile(df[column].values, 75)
    # IQR = Q3 - Q1
    # IQR_weight  = IQR * 1.5
    # minimum     = Q1 - IQR_weight
    # maximum     = Q3 + IQR_weight
    # outlier_idx = df[column][  (df[column] < minimum) | (df[column] > maximum)  ].index

# outlier_idx = CHECK_OUTLIER(df=df, column=col)
# print(col, oulier_idx)
# df.drop(outlier_idx, axis=0, inplace=True)
    #---------------------------------------------
    # numeric_columns = df.dtypes[df.dtypes != 'object'].index
    # for i, col in enumerate(numeric_columns) :
    #     oulier_idx = CHECK_OUTLIER(df=df, column=col)
    #     print(col , oulier_idx)
    #     #df.drop(outlier_idx, axis=0, inplace=True)


def MyCheckZeros(X):
    print("Check Zeros --------------------------------")
    for col in X.columns:
        gcnt = X[col][X[col] == 0].count()
        print(col, ":", gcnt, "(", np.round(gcnt / X.shape[0] * 100, 2), ")")
    print(barline)

def MyCheckNull(X):
    print("Check Null ---------------------------------")
    # print(df.isnull().sum())
    for col in X.columns:
        gcnt = X[col][X[col] == np.nan].count()
        print(col, ":", gcnt, "(", np.round(gcnt / X.shape[0] * 100, 2), ")")
    print(barline)

def MyScores(y_test, pred, proba, str=None) :
    print("Scores : {} ---------" .format(str))
    acc       = accuracy_score(y_test, pred)
    f1        = f1_score(y_test, pred, average="macro")
    precision = precision_score(y_test, pred, average="macro")
    recall    = recall_score(y_test, pred, average="macro")
    # auc       = roc_auc_score(y_test, proba[:, 1].reshape(-1, 1), multi_class="ovo")
    print("정확도{:.4f}  F1 {:.4f}=(정밀도{:.4f} 재현률{:.4f}) " .format(acc, f1, precision, recall))  # auc {:.4f}, auc
    cf_matrix = confusion_matrix(y_test, pred)
    print("Confusion Matrix", e, cf_matrix, barline)
    return cf_matrix


def MySplitFitPredict(model, X, y, str=None, rate=0.2, prc=False, roc=False) :
    X_train ,X_test , y_train, y_test = train_test_split(X, y, test_size=rate, random_state=121)
    model.fit(X_train, y_train)
    pred  = model.predict(X_test)
    proba = model.predict_proba(X_test)

    my_matrix = MyScores(y_test, pred, proba, str)

    if prc == True:
        MyChartPrcRclCurve(y_test, proba)
    if roc == True:
        MyChartRocCurve(y_test, proba)
    return proba, my_matrix, pred # X_train, X_test, y_train, y_test, proba # , pred, proba
# rf_model = RandomForestClassifier()
# SPLIT_FIT_PREDICT(rf_model, X, y, str="최초점수", 0.5)
# SPLIT_FIT_PREDICT(rf_model, X, y)


def MyChartHeatmap(df, targetClass=None, topn=0):
    plt.figure(figsize=(10,6))
    if   topn > 0 :
        idx = df.corr().nlargest(topn, targetClass).index
        print("Correlation: Top", topn, e, idx, barline)
        sns.heatmap(df[idx].corr(), annot=True, fmt=".2g", cmap="Blues")
    elif topn < 0 :
        idx = df.corr().nsmallest(-1 * topn, targetClass).index
        print("Correlation: Bottom", -topn, e, idx, barline)
        sns.heatmap(df[idx].corr(), annot=True, fmt=".2g", cmap="Blues")
        # cols = ['V12', 'V14', 'V10', 'V17', 'V3','Class']
        # sns.heatmap(df[cols].corr(), annot=True, fmt=".2g", cmap="Blues")
    else :
        sns.heatmap(df.corr(), annot=True, fmt=".2g", cmap="Blues")
    plt.show()
# CHART_HEATMAP(df, 5)
# CHART_HEATMAP(df, -5)
# CHART_HEATMAP(df)

def MyCrossValidate(model, X, y, cv=5) :
    print("Cross Validation ---------------------------")
    my_score   = {"acc":"accuracy", "f1":"f1"}
    score_list = cross_validate(model, X, y, scoring=my_score, cv=5, verbose=0)
    score_df   = pd.DataFrame(score_list)
    print("cross_validation 평균 정확도 : " , score_df["test_acc"].mean())
    print("cross_validation 평균 f1    : "    , score_df["test_f1"].mean())
    print(barline)
# CROSS_VALIDATION(model, X, y)

my_hyper_param = {  "n_estimators"     :[100, 300],
                    "max_depth"        :[4, 5, 6],
                    "min_samples_leaf" :[2, 3, 4],
                    "random_state"     :[121,]    }

def MyGridSearchCV(my_hyper_param, model, X, y) :
    print("GridSerchCV --------------------------------")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=121)
    my_score  = {"acc": "accuracy", "f1": "f1"}
    gcv_model = GridSearchCV(model, param_grid=my_hyper_param, scoring=my_score, refit="f1", cv=5, verbose=0)
    gcv_model.fit(X_train, y_train)

    # print("best_estimator:", gcv_model.best_estimator_)
    print("best_params :", gcv_model.best_params_)
    print("best_score  :", gcv_model.best_score_)
    print("GridSearchCV 평균 정확도 : ", gcv_model.cv_results_["mean_test_acc"].mean())  #mean_test_(본인의score키값)
    print("GridSearchCV 평균 F1    : ", gcv_model.cv_results_["mean_test_f1"].mean())
    print(barline)
# my_hyper_param = {  "n_estimators"     :[100]}
# GRIDSEARCHCV(my_hyper_param, model, X_train, y_train)


def MyGroupFillNa(bycol, cols, tcol):
    dict = df.groupby(by=[bycol])[cols].mean().astype(np.int32).to_dict()
    fill_mean_func = lambda gname: gname.fillna(dict[bycol][gname[tcol]])
    df   = df.groupby(bycol).apply(fill_mean_func)
    return df


def MyReplace(target_df, a, b) :
    target_df = target_df.replace(a, b)
    return target_df


def MyScaler(X):
    scaler   = StandardScaler()
    X_scaler = scaler.fit_transform(X)
    return X_scaler
# X_scaler = SCALER(StandardScaler())

def MyChartPrcRclCurve(y_test, proba):
    precision, recall, th = precision_recall_curve(y_test, proba[:, 1])
    print(len(precision), len(recall), len(th), barline)
    plt.plot(th, precision[:len(th)], label="precision")
    plt.plot(th, recall[:len(th)],    label="recall")
    plt.xlabel("threadshold"); plt.ylabel("precision & recall value")
    plt.legend()  # plt.legend(["precision","recall"])
    plt.grid()
    plt.show()

def MyChartRocCurve(y_test, proba):
    fpr, tpr, th = roc_curve(y_test, proba[:, 1])
    plt.plot(fpr, tpr, label='ROC')
    plt.plot([0, 1], [0, 1], label='th:0.5')
    # plt.title(auc)
    plt.xlabel("FPR"); plt.ylabel("TPR(recall)")
    plt.grid()
    plt.show()


# ------------------------------------------------
# precision_recall_curve : 임계치 튜닝을 통한 점수 보정
#-------------------------------------------------
ths = [.4, .43, .45, .47, .49, .51, .53]
def MyPrcRclTune(model, X, y, ths=ths):
    for th in ths:
        print("N : P =", th, ":", 1 - th)
        prob = MySplitFitPredict(model, X, y, prc=True)
        print(prob[4])
        bn = Binarizer(threshold=ths)
        fit_trans = bn.fit_transform(prob[:, 1].reshape(-1, 1))  # 세로로 reshape

        auc = roc_auc_score(y_test, prob[:, 1].reshape(-1, 1))
        print(auc)

# ------------------------------------------------
# boxplot : Boston-house-prices 사용
# distplot: Boston-house-prices 사용
#-------------------------------------------------
def MyPlots(data, plot, ncols=7, nrows=2):
    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(10, 10))
    index = 0
    axs = axs.flatten()     # 1차원으로 바꿔주기
    for k, v in data.items():
        if   plot == 'distplot':
            sns.distplot(v, ax=axs[index])
        elif plot == 'boxplot':
            sns.boxplot(y=k, data=data, ax=axs[index])
        index += 1
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
# df_feature = pd.read_csv("train_features.csv")
# df_labels  = pd.read_csv("train_labels.csv")

In [ ]:
# df_feature.info()
# df_labels.info()

In [ ]:
# df = pd.merge(df_feature, df_labels, on='id', how='outer')
# df.head(100)

In [ ]:
# target_col = ["label", "label_desc"]
# X_train = df.drop(target_col, axis=1)
# y_train = df["label"]

# X_train.info()
# y_train.head()

## 1. 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# train       = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/train_features.csv')
# train_label = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/train_labels.csv')
# test        = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/test_features.csv')
# submission  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/sample_submission.csv')

train       = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_features.csv')
train_label = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_labels.csv')
test        = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_features.csv')
submission  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')

## 2. 데이터 탐색

In [ ]:
# train.head(3)

In [ ]:
# submission.head(3)

## 3. 데이터 전처리

In [ ]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
X_train = train[features].groupby('id').agg(['median', 'std', 'max', 'min', 'var']) # 'max', 'min']) #, 'median', 'std'])
X_test  = test[features].groupby('id').agg([ 'median', 'std', 'max', 'min', 'var']) #'max', 'min' ]) #, 'median', 'std'])

In [ ]:
X_train.shape, X_test.shape

((3125, 30), (782, 30))

In [ ]:
y_train = train_label['label']

In [ ]:
y_train.head(3)

0    37
1    26
2     3
Name: label, dtype: int64

In [ ]:
#컬럼명 변경
p_columns = [agg + '_' + column for agg, column in X_train.columns]
X_train.columns = p_columns
X_test.columns = p_columns

In [ ]:
X_train.head(1)

,acc_x_median,acc_x_std,acc_x_max,acc_x_min,acc_x_var,acc_y_median,acc_y_std,acc_y_max,acc_y_min,acc_y_var,acc_z_median,acc_z_std,acc_z_max,acc_z_min,acc_z_var,gy_x_median,gy_x_std,gy_x_max,gy_x_min,gy_x_var,gy_y_median,gy_y_std,gy_y_max,gy_y_min,gy_y_var,gy_z_median,gy_z_std,gy_z_max,gy_z_min,gy_z_var
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.956149,0.191479,1.344268,0.59194,0.036664,-0.240638,0.177131,0.176871,-0.624113,0.031375,-0.346749,0.135131,-0.054876,-0.786336,0.01826,-1.273569,13.284216,31.644123,-46.254836,176.470384,-2.36223,24.300479,69.847244,-85.887677,590.513292,1.913286,25.275185,55.953827,-79.930029,638.834979


In [ ]:
X_test.head(1)

,acc_x_median,acc_x_std,acc_x_max,acc_x_min,acc_x_var,acc_y_median,acc_y_std,acc_y_max,acc_y_min,acc_y_var,acc_z_median,acc_z_std,acc_z_max,acc_z_min,acc_z_var,gy_x_median,gy_x_std,gy_x_max,gy_x_min,gy_x_var,gy_y_median,gy_y_std,gy_y_max,gy_y_min,gy_y_var,gy_z_median,gy_z_std,gy_z_max,gy_z_min,gy_z_var
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3125,-1.064222,0.236232,-0.275446,-1.564,0.055806,-0.005735,0.091641,0.22804,-0.470937,0.008398,-0.268442,0.174672,0.286182,-0.573836,0.03051,-3.77015,12.897967,56.953059,-50.429364,166.357553,0.108956,31.993022,96.185341,-81.607713,1023.553453,-1.607847,12.251648,49.981455,-35.446915,150.102867


### plot 보기 - 시간별 동작 그래프

In [ ]:
def plot_acc_id(df, id_num):
    return df.query('id == @id_num')[['time', 'acc_x', 'acc_y', 'acc_z']].plot(x = 'time')

def plot_gy_id(df, id_num):
    return df.query('id == @id_num')[['time', 'gy_x', 'gy_y', 'gy_z']].plot(x = 'time')

# plot_acc_id(train, 0)
# plt.ylim((-1.5, 1.5))
# plot_gy_id(train, 0)
# plt.ylim((-100, 100))

# plot_acc_id(train, 3)
# plt.ylim((-1.5, 1.5))
# plot_gy_id(train, 3)
# plt.ylim((-200, 200))

쉴 때는 움직임이 dynamic하지 않음
운동과 쉴 때 측정된 값 높낮이가 다를 것이며, 운동 끼리도 높낮이가 다를 것으로 추정

## 4. 모델링

### RandomForest

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# rf_model = RandomForestClassifier(random_state=121, min_samples_leaf=30)

# rf_prob = MySplitFitPredict(rf_model, X_train, y_train)

In [ ]:
rf_prob[0].shape

(625, 61)

In [ ]:
rf_hyper_param = {  "n_estimators"     :[100, 300],
                    "max_depth"        :[4, 5, 6],
                    "min_samples_leaf" :[2, 3, 4],
                    "random_state"     :[121,]    }

### XGBoost

In [ ]:
# GridSearchCV
xg_param_grid={'booster'  :['gbtree'],
            'silent'   :[True],
            'max_depth':[3, 5, 7],
            'min_child_weight':[1, 3, 5],
            'gamma'    :[0, 1, 2, 3],
            'nthread'  :[4],
            'objective':['multi:softmax'],
            'eval_metric'      :['logloss'],
            'colsample_bytree' :[0.5, 0.8],
            'colsample_bylevel':[0.9],
            'n_estimators'     :[50],
            'random_state'     :[2]}       # , scale_pos_weight: []

# XGBoost
xg_params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'eval_metric': 'logloss',
    'eta' : 0.02,
    'lambda': 2.0,
    'alpha': 1.0,
    'lambda_bias': 6.0,
    'num_class': 190625 ,   # 3125 * 61
    'n_jobs' : 4,
    'silent': 1,
}

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance

# xg_df = pd.DataFrame()
# # xg_df["train"] = X_train.values

# xg_train = xgb.DMatrix(X_train.values, label = y_train.values)
# xg_test  = xgb.DMatrix(X_test.values)

# wlist = [(xg_train, 'XDtrain'), (xg_test, 'XGeval')]
# xgb_model = xgb.train(params = xg_params, dtrain = xg_train, num_boost_round = 400, evals = wlist)

In [ ]:
xg_model = XGBClassifier()

# plot_importance(xg_model)
# plt.show()

xg_prob = MySplitFitPredict(xg_model, X_train, y_train)

Scores : None ---------
정확도0.7600  F1 0.5264=(정밀도0.5973 재현률0.5074) 
Confusion Matrix 
 [[0 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 4 0]
 [0 0 0 ... 0 0 7]] 


In [ ]:
xg_prob[0].shape

(625, 61)

### Scaler
- XGBoost랑 같이 써도 변화 X

In [ ]:
# X_train Scale: 합친 거
scaler   = RobustScaler()

X_scaler = scaler.fit_transform(X_train)

# X_train_col = ['acc_x_max', 'acc_x_min', 'acc_x_median', 'acc_x_std', 'acc_y_max', 'acc_y_min', 'acc_y_median', 'acc_y_std', 'acc_z_max', 'acc_z_min', 'acc_z_median', 'acc_z_std', 'gy_x_max', 'gy_x_min', 'gy_x_median', 'gy_x_std', 'gy_y_max', 'gy_y_min', 'gy_y_median', 'gy_y_std', 'gy_z_max', 'gy_z_min', 'gy_z_median', 'gy_z_std']
# X_train_col = ['acc_x_max', 'acc_x_min', 'acc_y_max', 'acc_y_min', 'acc_z_max', 'acc_z_min', 'gy_x_max', 'gy_x_min', 'gy_y_max', 'gy_y_min', 'gy_z_max', 'gy_z_min']
X_train_col = ['acc_x_median', 'acc_x_std', 'acc_x_max', 'acc_x_min', 'acc_x_val', 'acc_y_median', 'acc_y_std', 'acc_y_max', 'acc_y_min', 'acc_y_val', 'acc_z_median', 'acc_z_std', 'acc_z_max', 'acc_z_min', 'acc_z_val', 'gy_x_median', 'gy_x_std','gy_x_max', 'gy_x_min', 'gy_x_val', 'gy_y_median', 'gy_y_std', 'gy_y_max', 'gy_y_min', 'gy_y_val', 'gy_z_median', 'gy_z_std', 'gy_z_max', 'gy_z_min','gy_z_val']

X_scale_df  = pd.DataFrame(X_scaler, columns = X_train_col)

In [ ]:
scale_idx = pd.DataFrame(np.arange(0, 3125, 1).reshape(-1, 1))
scale_idx.columns = ['id']
scale_idx.info()
print(X_scale_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3125 entries, 0 to 3124
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      3125 non-null   int64
dtypes: int64(1)
memory usage: 24.5 KB
(3125, 30)


In [ ]:
# X_scale_df = pd.concat([X_scale_df,scale_idx], axis=1)
# X_scale_df.set_index('id', inplace=True)
# X_scale_df.info()
X_scale_df.head(3)

,acc_x_median,acc_x_std,acc_x_max,acc_x_min,acc_x_val,acc_y_median,acc_y_std,acc_y_max,acc_y_min,acc_y_val,acc_z_median,acc_z_std,acc_z_max,acc_z_min,acc_z_val,gy_x_median,gy_x_std,gy_x_max,gy_x_min,gy_x_val,gy_y_median,gy_y_std,gy_y_max,gy_y_min,gy_y_val,gy_z_median,gy_z_std,gy_z_max,gy_z_min,gy_z_val
0,2.006536,-0.415878,0.727936,2.824735,-0.318507,-0.019853,-0.337247,-0.344720,0.384740,-0.282603,-0.868320,-0.645694,-1.307345,-0.225411,-0.500846,0.020751,-0.915798,-0.798702,0.727013,-0.612525,-0.384399,-0.513259,-0.488766,0.375606,-0.380359,0.403676,-0.371013,-0.473053,0.282853,-0.290053
1,-0.156220,0.425633,0.631714,-0.964562,0.511855,0.000926,0.474854,0.195710,-0.375282,0.548676,-0.146101,1.445823,0.304999,-0.574036,2.188891,0.682123,1.009845,0.856025,-1.314954,1.444380,1.622385,1.297867,1.697493,-1.408375,1.962455,-0.143650,0.937114,1.536133,-1.231552,1.300905
2,1.005530,1.024682,0.619334,0.432715,1.550806,0.295458,-0.490218,0.144661,0.309046,-0.381472,1.402519,0.007006,1.064286,1.060552,0.007048,-0.868888,-0.561421,-0.526910,-0.140223,-0.454175,3.794343,1.877574,1.066510,-0.901360,3.302356,0.756389,-0.666488,-0.475251,0.566138,-0.429760


In [ ]:
scaler   = RobustScaler()
X_test_scaler = scaler.fit_transform(X_test)
X_test_df  = pd.DataFrame(X_test_scaler, columns = X_train_col)

scale_idx = pd.DataFrame(np.arange(0, 3125, 1).reshape(-1, 1))
scale_idx.columns = ['id']
scale_idx.info()
print(X_test_df.shape)

X_test_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3125 entries, 0 to 3124
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      3125 non-null   int64
dtypes: int64(1)
memory usage: 24.5 KB
(782, 30)


,acc_x_median,acc_x_std,acc_x_max,acc_x_min,acc_x_val,acc_y_median,acc_y_std,acc_y_max,acc_y_min,acc_y_val,acc_z_median,acc_z_std,acc_z_max,acc_z_min,acc_z_val,gy_x_median,gy_x_std,gy_x_max,gy_x_min,gy_x_val,gy_y_median,gy_y_std,gy_y_max,gy_y_min,gy_y_val,gy_z_median,gy_z_std,gy_z_max,gy_z_min,gy_z_val
0,-0.444766,-0.309577,-0.653016,-0.132973,-0.258417,0.433632,-0.735930,-0.248245,0.569126,-0.495352,-0.665102,-0.391443,-0.722800,0.077095,-0.324618,-0.268476,-0.901758,-0.647715,0.733813,-0.574599,0.088315,-0.299418,-0.304360,0.467561,-0.240068,-0.273030,-0.666445,-0.506536,0.639222,-0.415932
1,0.017370,0.538745,0.141953,-0.630641,0.683710,1.524605,0.525795,1.285218,0.887853,0.617511,-0.105279,0.857431,-0.107817,-1.004315,1.069460,-1.610304,0.222715,-0.135064,-0.858974,0.238583,-0.111896,0.033069,0.695789,0.345961,0.031276,-2.248240,0.602012,0.359815,-0.317143,0.743388
2,1.420918,-0.355140,2.205922,0.918286,-0.288166,2.104052,-0.214021,1.526601,1.382807,-0.188440,-0.406860,-0.526377,-0.152019,0.228276,-0.412745,0.272376,-0.615091,0.051381,0.381407,-0.459997,-0.360297,-0.748571,-0.453667,0.335599,-0.454535,-0.589874,-0.389129,-0.163943,-0.375808,-0.294815


In [ ]:
# X 원본 데이터 scale
X      = tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
scaler = RobustScaler()

num_sample   = X.shape[0] # 3 (샘플 데이터)
num_sequence = X.shape[1] # 4 (시계열 데이터)
num_feature  = X.shape[2] # 2 (Feature)

# 시계열을 선회하면서 피팅합니다
for ss in range(num_sequence):
    scaler.fit_transform(X[:, ss, :])
# 스케일링(변환)합니다.
results = []
for ss in range(num_sequence):
    results.append(scaler.transform(X[:, ss, :]).reshape(num_sample, 1, num_feature))
X_scaled = np.concatenate(results, axis=1)


# X_scaler = scaler.fit_transform(X)

# X_scaled.shape

In [ ]:
from sklearn.metrics import make_scorer

def MyGridSearchCV(my_hyper_param, grid_model, X_grid, y_grid) :
    print("GridSerchCV --------------------------------")
    X_train_grid, X_test_grid, y_train_grid, y_test_grid = train_test_split(X_grid, y_grid, test_size=0.2, random_state=121)
    # my_score  = {"acc": "accuracy", "f1": "f1"}
    gcv_model = GridSearchCV(grid_model, param_grid=my_hyper_param, scoring=make_scorer(f1_score, average='micro'), refit="f1", cv=5, verbose=0, return_train_score=True)
    gcv_model.fit(X_train_grid, y_train_grid)
    gcv_y_pred = gcv_model.predict(X_test)

    # print("best_estimator:", gcv_model.best_estimator_)
    print("best_params :", gcv_model.best_params_)
    print("best_score  :", gcv_model.best_score_)
    # print("GridSearchCV 평균 정확도 : ", gcv_model.cv_results_["mean_test_acc"].mean())  # mean_test_(본인의score키값)
    print("GridSearchCV 평균 F1     : ", gcv_model.cv_results_["mean_test_score"].mean())
    print(barline)

MyGridSearchCV(rf_hyper_param, rf_model, X_train, y_train)

GridSerchCV --------------------------------


KeyboardInterrupt: ignored

## Dobby

In [ ]:
# import pandas as pd
# import numpy as np
# import random
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# from glob import glob

# x_train = []

# for uid in tqdm(train['id'].unique()):
#     temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
#     x_train.append(temp)

# x_train = np.array(x_train, np.float32)
# x_train = x_train[:,:,:,np.newaxis]

# x_test = []

# for uid in tqdm(test['id'].unique()):
#     temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
#     x_test.append(temp)

# x_test = np.array(x_test, np.float32)
# x_test = x_test[:,:,:,np.newaxis]

In [ ]:
# x_train.shape

## DACON

In [ ]:
# X, y 원본
X      = tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
test_X = tf.reshape(np.array( test.iloc[:,2:]),[-1, 600, 6])
y      = tf.keras.utils.to_categorical(y_train) 

# X_train, y_train aggregation
# X_train = train[features].groupby('id').agg(['max', 'min', 'median', 'std'])   #, 'median', 'std'])
# X_test  = test[features].groupby('id').agg(['max', 'min', 'median', 'std'])    #, 'median', 'std'])
# y_train = train_label['label']
print(X.shape, y.shape)

(3125, 600, 6) (3125, 61)


### 모델 학습 및 예측 - normal

In [ ]:
model = Sequential()

# model.add(LSTM(32, input_shape=(600,6)))                    # 합치지 않은 데이터 사용시(시계열 유지)
model.add(Dense(30,  input_dim=12, activation='relu'))    # 합친 데이터 또는 time 없이 쓰기

model.add(Dense(20, activation='relu'))
model.add(Dense(12, activation='relu'))
# Dropout(0.25)
model.add(Dense(8 , activation='relu'))
model.add(Dense(61, activation='softmax'))                  # sigmoid or softmax

model.compile(loss='categorical_crossentropy',              # binary_crossentropy or categorical_crossentropy
              optimizer='adam', metrics=['accuracy'])       # optimizer='rmsprop' : basecode # adam이 상위

# 모델 업데이트 및 저장
modelpath="content/MyDrive/Colab Notebooks/dacon/model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

# 학습: epoch - 역전파 횟수, batch_size - 나눠 돌리는 수
  # X값 바꿔서 해보기(X_scaler, X_train, etc.), y는 그대로 두기
model.fit(X_scaler, y, epochs=3000, batch_size=100, validation_split=0.2, verbose=0, callbacks=[early_stopping_callback,checkpointer])

## prediction & 저장
- model: rf_model, xg_model, etc.

In [ ]:
xg_model.fit(X_scale_df, y_train)

# X 원본 시계열 데이터 사용했을 때, test_X 사용
# prediction = xg_model.predict(test_X)

# X_train 합친 데이터(Max, Min, etc.) 사용했을 때, X_test 사용
prediction = xg_model.predict(X_test_df)
proba = xg_model.predict_proba(X_test_df)

# test_X.shape
# X_test.shape
print(prediction.shape, proba.shape)

(782,) (782, 61)


In [ ]:
# submission.iloc[:,1:]=prediction      # sequential 사용시
submission.iloc[:,1:]=proba             # xg_model, rf_model 등
submission.to_csv('baseline_submission1111.csv', index=False)

In [ ]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000733,0.000219,0.000247,0.000418,0.000947,0.000191,0.000251,0.000295,0.000343,0.021629,0.234103,0.047584,0.000635,0.549483,0.004767,0.001279,0.000219,0.000211,0.000198,0.000218,0.000248,0.000337,0.000639,0.001138,0.011116,0.000204,0.027362,0.000318,0.000244,0.000243,0.010572,0.001072,0.001055,0.000309,0.003164,0.000212,0.009875,0.000251,0.000806,0.000397,0.000191,0.000208,0.000376,0.000259,0.000217,0.000355,0.000210,0.000274,0.001145,0.002877,0.000427,0.009054,0.000908,0.001085,0.000560,0.000359,0.000197,0.000367,0.045198,0.000211,0.001990
1,3126,0.001002,0.000281,0.000394,0.001098,0.001414,0.000308,0.000273,0.000782,0.000643,0.000579,0.000529,0.000401,0.000291,0.000607,0.000283,0.000944,0.000577,0.000284,0.000257,0.000293,0.000314,0.002202,0.002280,0.005566,0.000793,0.000297,0.940768,0.000319,0.000253,0.000276,0.000441,0.000582,0.000619,0.000506,0.015462,0.000756,0.000444,0.000304,0.000317,0.000313,0.001836,0.000870,0.000359,0.000726,0.000357,0.000312,0.000276,0.000435,0.000890,0.001618,0.000672,0.000440,0.000275,0.000287,0.000402,0.000494,0.000265,0.000563,0.000379,0.000330,0.004160
2,3127,0.002785,0.112253,0.000613,0.002376,0.001029,0.000802,0.001576,0.000928,0.003229,0.000844,0.001271,0.001116,0.000617,0.000790,0.000659,0.001893,0.018932,0.005657,0.000557,0.000621,0.000617,0.001506,0.001848,0.000737,0.001438,0.000667,0.394983,0.004667,0.000776,0.000949,0.001080,0.000698,0.003706,0.039459,0.009018,0.000810,0.004685,0.001002,0.000739,0.001094,0.020640,0.002866,0.000675,0.001862,0.001650,0.004354,0.000665,0.150902,0.004863,0.005721,0.000874,0.000982,0.000928,0.000863,0.007282,0.000713,0.008753,0.000784,0.000956,0.141451,0.013189
3,3128,0.036586,0.000538,0.000829,0.000647,0.000909,0.000697,0.000571,0.000856,0.000633,0.000470,0.000678,0.000656,0.000624,0.000757,0.000619,0.000783,0.000655,0.000632,0.000552,0.000614,0.000598,0.107649,0.000946,0.001726,0.001077,0.000573,0.712597,0.000576,0.000679,0.000579,0.001522,0.000925,0.004506,0.000559,0.001245,0.000732,0.001833,0.000722,0.000659,0.000605,0.000571,0.000557,0.002164,0.002349,0.000574,0.000559,0.000592,0.000618,0.006204,0.043867,0.011545,0.000734,0.000575,0.000600,0.001327,0.000550,0.000555,0.007348,0.000661,0.000593,0.027845
4,3129,0.002611,0.000089,0.000151,0.000259,0.000229,0.000089,0.000090,0.000181,0.000320,0.000407,0.000099,0.000136,0.000098,0.000156,0.000118,0.000198,0.000078,0.000103,0.000902,0.000096,0.000105,0.000163,0.002786,0.000971,0.000094,0.000090,0.979223,0.000090,0.000135,0.000095,0.000155,0.000081,0.000522,0.000081,0.003789,0.000702,0.000159,0.000106,0.000111,0.000092,0.000176,0.000101,0.000164,0.000081,0.000094,0.000095,0.000091,0.000104,0.000285,0.001178,0.000579,0.000105,0.000090,0.000094,0.000090,0.000084,0.000087,0.000147,0.000102,0.000106,0.000187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.002465,0.000097,0.000118,0.000183,0.004050,0.000076,0.000062,0.000096,0.000528,0.000157,0.000125,0.000481,0.000068,0.000073,0.001332,0.000593,0.000066,0.000108,0.000504,0.000067,0.000152,0.000162,0.000473,0.000150,0.000558,0.000084,0.973059,0.000086,0.000157,0.000063,0.000460,0.000114,0.004341,0.000068,0.001465,0.000224,0.000204,0.000073,0.000123,0.000066,0.000164,0.000125,0.000319,0.000293,0.000072,0.000070,0.000064,0.000068,0.001997,0.001144,0.000611,0.000415,0.000117,0.000178,0.000086,0.000058,0.000130,0.000163,0.000163,0.000075,0.000358
778,3903,0.000293,0.000127,0.000128,0.000205,0.000185,0.000259,0.000125,0.000181,0.000104,0.000117,0.000116,0.000093,0.000103,0.000135,0.000100,0.000903,0.000264,0.000300,0.000182,0.000101,0.001077,0.000125,0.000350,0.000401,0.000260

129	eruru19 1.07384	